In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [ ]:
# import audit-data.csv
df = pd.read_csv('audit-data.csv', keep_default_na=False)
df

In [ ]:
# show the distribution of degree in a table
df.groupby(['degree']).size()

In [ ]:
# create bar plot for degree in percentage
degree = ['Bachelors', 'Master\'s', 'PhD']

temp_df_degree = df.groupby(by='degree').size()/df.groupby(by='degree').size().sum()
temp_df_degree.plot(kind='bar')

# rotate the x-axis labels
plt.xticks(rotation=0)

# add title and labels
plt.title('Distribution of Degree')
plt.xlabel('Degree')
plt.ylabel('Percentage of Samples')

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

# save as png
plt.savefig('degree.png', dpi=300)


In [ ]:
df.groupby(['disability']).size()

In [ ]:
df.groupby(['work_auth']).size()

In [ ]:
# show the distribution of ethnicity and gender and do not show the column names
df.groupby(['gender', 'ethinicity']).size().unstack()

In [ ]:
# show the distribution of GPA density in a histogram, remove grid and add n_bins
plt.hist(df['gpa'], weights=np.ones(len(df['gpa'])) / len(df['gpa']), color='tab:orange', alpha=0.7, edgecolor='tab:orange', bins=20)

# plt.hist(kind='hist', density=True, grid=False, bins=11)

# # add title and labels
plt.title('Distribution of GPA')
plt.xlabel('GPA')
plt.ylabel('Percentage of Samples')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

# # save as png
# plt.savefig('gpa.png', dpi=300)

In [ ]:
# show the distribution of mean resume score for each group_idx in a histogram, remove grid and add n_bins
n_bins = 20

plt.hist(df[df['gender']=='M'].groupby(['group_idx'])['resume_score'].mean(), n_bins, alpha=0.5, label='Male')
plt.hist(df[df['gender']=='F'].groupby(['group_idx'])['resume_score'].mean(), n_bins, alpha=0.5, label='Female')
plt.hist(df[df['gender']=='N/A'].groupby(['group_idx'])['resume_score'].mean(), n_bins, alpha=0.5, label='N/A')

plt.legend(loc='upper right')
plt.show()
